In [507]:
# Standard library imports
from datetime import datetime
import os

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Visualize loop progress

# Set pandas display options for better data frame visualization
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Segreti
from dotenv import load_dotenv
load_dotenv()

# API
import requests
import json

| Gap-Up Screener |
|--|
TradingView Data (import)

In [508]:
# today's date string for file naming automation
today_date_str = datetime.now().strftime("_%Y-%m-%d.csv")
# today_date_str = '_2024-02-29.csv'

print(today_date_str)

_2024-03-05.csv


In [509]:
# concat base file name with today's date string
filename = f"trading_view_raw_data/tv_screen_gap-up{today_date_str}"
# read in trading view raw data
trading_view_raw_df = pd.read_csv(filename)

print(f'{len(trading_view_raw_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
trading_view_raw_df.head(5)

834 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower"
0,NVDA,NVIDIA Corporation,NASDAQ,2130924987792.9995,USD,852.37,USD,853.390,USD,3.43,USD,0.402407,0.119666,2399495000,61561645,61561645,1102244,5.980448e+07,51889597.33333332,45671277.34444457,6.469260,3.159478,3.651927,855.503333,71.40870439,Electronic Technology,2.77518493,1.8023118,2.3419523,1.7245569,1.275180,1.582042,2.371919,2.295911,1.181824,0.670416,1.054314,1.050512,0.27511126,0.10877948,876.95000,USD,876.95000,USD,876.95000,USD,876.95000,USD,876.95000,USD,853.3199,USD,857.7899,USD,868.33,USD,868.33,USD,868.33,USD,876.95,USD,876.95,USD,876.95,USD,876.95,USD,2058175.67567568,USD,851.840,851.473125,855.28506,863.862000,862.319449,854.377825,846.436201,883.240852,867.361025,851.481198
1,V,Visa Inc.,NYSE,563575546147,USD,280.53,USD,279.520,USD,0.27,USD,0.096246,-0.360033,1576963849.25,5920308,5920308,1286,5.000700e+06,5431500.66666667,5690891.52222222,1.808507,1.268974,1.172140,280.553333,32.79786749,Commercial Services,0.93115891,0.7562648,0.8830137,0.96204925,4.799943,5.189779,4.315754,3.235009,1.771580,1.009361,1.163038,1.167312,0.24769875,0.08191545,286.13000,USD,286.13000,USD,286.13000,USD,286.13000,USD,286.13000,USD,280.6000,USD,280.6000,USD,280.77,USD,280.77,USD,280.77,USD,280.77,USD,283.00,USD,283.00,USD,284.91,USD,1133784.72222222,USD,280.230,280.197500,280.28606,280.383215,280.595558,280.259955,279.924352,280.904735,280.258905,279.613075
2,WMT,Walmart Inc.,NYSE,477530587947.9999,USD,59.30,USD,59.250,USD,0.28,USD,0.472175,-0.084317,4308838556.46,16033298,16033298,57369,3.454992e+07,23300434.76881788,23501971.71182818,1.695205,1.217253,1.285189,59.030000,31.01626654,Retail Trade,1.44614868,0.40185887,0.5880075,0.497185,3.816648,6.619298,6.805627,4.909827,1.883975,0.887682,0.839668,0.455699,0.16066981,0.08497213,60.44994,USD,60.44994,USD,60.44994,USD,60.44994,USD,60.44994,USD,59.3900,USD,59.3900,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,-,-,59.345,59.336250,59.28200,59.243077,59.380682,59.296250,59.211818,59.307319,59.235000,59.162681
3,JNJ,Johnson & Johnson,NYSE,385017361843,USD,159.84,USD,160.140,USD,0.29,USD,0.181431,0.187688,2404865383.83,8533083,8533083,1290,7.243365e+06,7624227.6,705731

In [510]:
def categorize_market_cap(df):
    # convert 'Market capitalization' to numeric, coercing errors to NaN
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    # define conditions for market cap categories
    conditions = [
        # Titans
        (df['Market capitalization'] >= 200000000000),  # 200 billion and above
        #Large Cap
        (df['Market capitalization'] >= 10000000000) & (df['Market capitalization'] < 200000000000),  # 10 billion to 200 billion
        # Midlers
        (df['Market capitalization'] >= 2000000000) & (df['Market capitalization'] < 10000000000),  # 2 billion to 10 billion
        # Small Cap
        (df['Market capitalization'] >= 300000000) & (df['Market capitalization'] < 2000000000),  # 300 million to 2 billion
        # Micro Cap
        (df['Market capitalization'] > 50000000) & (df['Market capitalization'] < 300000000),  # 300 million and below (excluding 50 million and below)
        # Shrimp
        (df['Market capitalization'] <= 50000000)  # 50 million and below
    ]
    categories = ['Titans', 'Large caps', 'Midlers', 'Small caps', 'Micro caps', 'Shrimp']
    # use np.select to assign categories based on conditions
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    
    return df


In [511]:
# EXECUTE
trading_view_raw_df = categorize_market_cap(trading_view_raw_df)
trading_view_raw_df.head(5)

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,NVDA,NVIDIA Corporation,NASDAQ,2.130925e+12,USD,852.37,USD,853.390,USD,3.43,USD,0.402407,0.119666,2399495000,61561645,61561645,1102244,5.980448e+07,51889597.33333332,45671277.34444457,6.469260,3.159478,3.651927,855.503333,71.40870439,Electronic Technology,2.77518493,1.8023118,2.3419523,1.7245569,1.275180,1.582042,2.371919,2.295911,1.181824,0.670416,1.054314,1.050512,0.27511126,0.10877948,876.95000,USD,876.95000,USD,876.95000,USD,876.95000,USD,876.95000,USD,853.3199,USD,857.7899,USD,868.33,USD,868.33,USD,868.33,USD,876.95,USD,876.95,USD,876.95,USD,876.95,USD,2058175.67567568,USD,851.840,851.473125,855.28506,863.862000,862.319449,854.377825,846.436201,883.240852,867.361025,851.481198,Titans
1,V,Visa Inc.,NYSE,5.635755e+11,USD,280.53,USD,279.520,USD,0.27,USD,0.096246,-0.360033,1576963849.25,5920308,5920308,1286,5.000700e+06,5431500.66666667,5690891.52222222,1.808507,1.268974,1.172140,280.553333,32.79786749,Commercial Services,0.93115891,0.7562648,0.8830137,0.96204925,4.799943,5.189779,4.315754,3.235009,1.771580,1.009361,1.163038,1.167312,0.24769875,0.08191545,286.13000,USD,286.13000,USD,286.13000,USD,286.13000,USD,286.13000,USD,280.6000,USD,280.6000,USD,280.77,USD,280.77,USD,280.77,USD,280.77,USD,283.00,USD,283.00,USD,284.91,USD,1133784.72222222,USD,280.230,280.197500,280.28606,280.383215,280.595558,280.259955,279.924352,280.904735,280.258905,279.613075,Titans
2,WMT,Walmart Inc.,NYSE,4.775306e+11,USD,59.30,USD,59.250,USD,0.28,USD,0.472175,-0.084317,4308838556.46,16033298,16033298,57369,3.454992e+07,23300434.76881788,23501971.71182818,1.695205,1.217253,1.285189,59.030000,31.01626654,Retail Trade,1.44614868,0.40185887,0.5880075,0.497185,3.816648,6.619298,6.805627,4.909827,1.883975,0.887682,0.839668,0.455699,0.16066981,0.08497213,60.44994,USD,60.44994,USD,60.44994,USD,60.44994,USD,60.44994,USD,59.3900,USD,59.3900,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,59.39,USD,-,-,59.345,59.336250,59.28200,59.243077,59.380682,59.296250,59.211818,59.307319,59.235000,59.162681,Titans
3,JNJ,Johnson & Johnson,NYSE,3.850174e+11,USD,159.84,USD,160.140,USD,0.29,USD,0.181431,0.187688,2404865383.83,8533083,8533083,1290,7.243

In [512]:
# drop NaNs
tv_df = trading_view_raw_df.dropna()
print(f'NaNs = {tv_df.isna().sum().sum()}')

# confirm no NaNs
if tv_df.isna().sum().sum() == 0:
    nan_count = tv_df.isna().sum().sum()
    print(f"There are zero {nan_count} NaNs in DataFrame! Nice work!")
else:
    print(f"There are {tv_df.isna().sum().sum()} NaNs in DataFrame, check your code dumb dumb!")

NaNs = 0
There are zero 0 NaNs in DataFrame! Nice work!


In [513]:
# view the raw counts and percentages of each market cap type
market_cap_type_df = trading_view_raw_df['marketCapType'].value_counts().reset_index()
market_cap_type_df.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage -> add new columne to dataframe
market_cap_type_df['percentage'] = (market_cap_type_df['count'] / market_cap_type_df['count'].sum()) * 100

print(f'There are still the same {len(trading_view_raw_df.index)} U.S. Stocks in the dataframe \n$(pre-market change) > $0.00 USD')
market_cap_type_df

There are still the same 834 U.S. Stocks in the dataframe 
$(pre-market change) > $0.00 USD


,marketCapType,count,percentage
0,Shrimp,241,28.896882
1,Micro caps,167,20.023981
2,Small caps,165,19.784173
3,Large caps,128,15.347722
4,Midlers,112,13.429257
5,Titans,11,1.318945
6,Undefined,10,1.199041


In [514]:
# print length of df
print(f'{len(tv_df.index)} U.S. Stocks \n$(pre-market change) > $0.00 USD')
# print sample head
tv_df.sample(5)

824 U.S. Stocks 
$(pre-market change) > $0.00 USD


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
769,KWE,KWESST Micro Systems Inc.,NASDAQ,4.482047e+06,USD,1.1000,USD,1.1400,USD,0.0500,USD,4.545455,3.636364,3163169.41044,37408,37408,1000,157093.7,78426.6,52405.67777778,7.619048,9.044102,9.547242,1.093333,-,Commercial Services,-,-0.12600258,0.47772956,0.00079634,0.220093,0.219103,0.443959,0.403515,0.317869,0.165305,0.977361,0.238741,0.11554734,0.07533441,1.490,USD,1.6729,USD,1.920,USD,3.8491,USD,145.4600,USD,1.100,USD,1.1200,USD,1.1200,USD,1.1200,USD,1.1200,USD,1.1200,USD,1.1300,USD,1.1300,USD,1.1300,USD,31119.13167808,USD,1.09400,1.086250,1.09400,1.091538,1.137619,1.092510,1.047401,1.141186,1.094510,1.047834,Shrimp
804,PRSO,Peraso Inc.,NASDAQ,2.251091e+06,USD,1.7500,USD,1.7400,USD,0.0100,USD,0.571429,-0.571429,579173.908834,191027,191027,5757,1021910.8,510512.5,214440.57777778,8.284024,7.830207,15.073868,1.753333,-,Electronic Technology,1.76535423,1.6204829,1.5032433,1.8444726,6.574336,3.749699,3.210278,1.656272,0.699964,0.511765,0.846924,0.188281,0.12417419,0.12984647,3.890,USD,11.2000,USD,15.656,USD,52.8000,USD,182160.0000,USD,1.740,USD,1.7400,USD,1.7400,USD,1.7400,USD,1.7400,USD,1.7400,USD,1.8200,USD,1.8200,USD,1.8200,USD,203671.23287671,USD,1.73452,1.729075,1.72462,1.723062,1.747796,1.724930,1.702064,1.744297,1.725795,1.707293,Shrimp
245,VIAV,Viavi Solutions Inc.,NASDAQ,2.165565e+09,USD,9.7300,USD,9.7500,USD,0.2800,USD,2.877698,0.205550,219355485.45,871038,871038,4192,1135242.2,2296636.09999999,1984192.72222223,1.497161,1.901764,2.993958,9.748333,434.375,Technology Services,1.64048933,1.407686,0.9703847,0.9602283,4.246712,3.425223,5.523942,3.151394,1.158982,0.714396,0.773914,0.719565,0.093159,0.0479886,10.220,USD,10.3200,USD,10.320,USD,11.6454,USD,697.8081,USD,9.740,USD,9.7400,USD,9.7650,USD,9.7650,USD,9.7650,USD,9.7900,USD,9.8300,USD,9.8300,USD,9.8300,USD,307250,USD,9.73300,9.730000,9.73500,9.757600,9.747000,9.731000,9.715000,9.791273,9.757660,9.724047,Midlers
814,DRMA,"Dermata Therapeutics, Inc.",NASDAQ,1.572747e+06,USD,0.4444,USD,0.4603,USD,0.0159,USD,3.577858,3.577858,2830146.491708,74486,74486,356,107647.2,162802.53333333,710572.53333333,7.048562,6.019105,8.334620,0.440900,-,Health Technology,-,0.45797753,0.

In [515]:
# drop undefined
tv_df = tv_df[tv_df['marketCapType'] != 'Undefined']

market_cap_type_df2 = tv_df['marketCapType'].value_counts().reset_index()
market_cap_type_df2.columns = ['marketCapType', 'count']  # Rename columns for clarity

# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df2['percentage'] = (market_cap_type_df2['count'] / market_cap_type_df2['count'].sum()) * 100

market_cap_type_df2
print(market_cap_type_df2)


  marketCapType  count  percentage
0        Shrimp    241   29.247573
1    Micro caps    167   20.266990
2    Small caps    165   20.024272
3    Large caps    128   15.533981
4       Midlers    112   13.592233
5        Titans     11    1.334951


In [516]:
# 1/3 #### CONFIG DATA SECTION ####
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # Reduced to 0.2% for Titans
        "float_shares_outstanding_threshold": 1000000000,  # Increased to 1 billion shares
        "relative_volume_threshold": 1.2,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.03,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.001,  # Reduced to 0.1%
        "pre_market_vmap_drawdown_threshold": 0.003  # .3% drawdown from VWAP
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # Reduced to 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # Increased to 200 million shares
        "relative_volume_threshold": 1.3,  # Slightly more inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # Reduced to 0.5%
        "pre_market_vmap_drawdown_threshold": 0.004  # .4% drawdown from VWAP
    },
    "Midlers": {
        "pre_market_change_pct_threshold": 0.02,  # 2% for Mid caps
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .02,
        "pre_market_vmap_drawdown_threshold": 0.005  # .5% drawdown from VWAP
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .03,
        "pre_market_vmap_drawdown_threshold": 0.006  # 6% drawdown from VWAP
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .04,
        "pre_market_vmap_drawdown_threshold": 0.007  # 7% drawdown from VWAP
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": .05,
        "pre_market_gap_percentage_threshold": .05,
        "pre_market_vmap_drawdown_threshold": 0.008  # 8% drawdown from VWAP
    }
}

# 2/3 #### CORE FUNCTIONS ####
def filter_by_pre_market_change(df, change_pct_threshold):
    """Filter stocks by pre-market change percentage."""
    return df[df['Pre-market Change %'] >= change_pct_threshold]

def filter_by_float_shares(df, float_shares_threshold):
    """Filter stocks by float shares outstanding."""
    # create a copy to safely modify without affecting the original DataFrame
    modified_df = df.copy()
    modified_df['Float shares outstanding'] = pd.to_numeric(modified_df['Float shares outstanding'], errors='coerce')
    return modified_df[modified_df['Float shares outstanding'] <= float_shares_threshold]

def filter_by_relative_volume(df, relative_volume_threshold):
    """Filter stocks by relative volume."""
    df['Relative Volume 1 day'] = pd.to_numeric(df['Relative Volume 1 day'], errors='coerce')
    return df[df['Relative Volume 1 day'] >= relative_volume_threshold]

def filter_by_relative_volume_at_time(df, relative_volume_at_time_threshold):
    """Filter stocks by relative volume at a specific time."""
    # Convert 'Relative Volume at Time' to float
    df['Relative Volume at Time'] = pd.to_numeric(df['Relative Volume at Time'], errors='coerce')
    return df[df['Relative Volume at Time'] >= relative_volume_at_time_threshold]

def filter_by_pre_market_gap_percentage(df, pre_market_gap_percentage_threshold):
    """Filter stocks by pre-market gap percentage."""
    return df[df['Pre-market Gap %'] >= pre_market_gap_percentage_threshold]

def filter_by_price_near_vwap(df, pre_market_vmap_drawdown_threshold):
    """
    Filter stocks where the price is near the VWAP, considering the specified drawdown threshold.
    """
    df_filtered = df.copy()
    df_filtered['Price'] = pd.to_numeric(df_filtered['Price'], errors='coerce')
    df_filtered['Volume Weighted Average Price 1 day'] = pd.to_numeric(df_filtered['Volume Weighted Average Price 1 day'], errors='coerce')
    
    # Calculate the acceptable minimum price based on the VWAP drawdown threshold
    df_filtered['Min Price from VWAP'] = df_filtered['Volume Weighted Average Price 1 day'] * (1 - pre_market_vmap_drawdown_threshold)
    
    # Apply the filter
    df_filtered = df_filtered[df_filtered['Price'] >= df_filtered['Min Price from VWAP']]
    
    return df_filtered.drop(columns=['Min Price from VWAP'])

"""SPECIAL FILTERS"""
def filter_by_volatility(df):
    """Filter stocks based on recent volatility being higher than weekly and monthly averages."""
    # Ensure data types are correct; convert to numeric if necessary
    df['Volatility 1 day'] = pd.to_numeric(df['Volatility 1 day'], errors='coerce')
    df['Volatility 1 week'] = pd.to_numeric(df['Volatility 1 week'], errors='coerce')
    df['Volatility 1 month'] = pd.to_numeric(df['Volatility 1 month'], errors='coerce')
    filtered_df = df[
        (df['Volatility 1 day'] >= df['Volatility 1 week']) & 
        (df['Volatility 1 day'] >= df['Volatility 1 month'])]
    return filtered_df

# 3/3 #### WRAPPER FUNCTION ####
def screen_stocks_by_category(df, category):
    """Screen stocks based on their market cap category."""
    # 3a. Get the configuration for the specified category
    config = criteria_config.get(category, {})
    pre_market_change_pct_threshold = config.get("pre_market_change_pct_threshold", 0)
    float_shares_threshold = config.get("float_shares_outstanding_threshold", float('inf'))
    relative_volume_threshold = config.get("relative_volume_threshold", 0)
    relative_volume_at_time_threshold = config.get("relative_volume_at_time_threshold", 0)
    pre_market_gap_percentage_threshold = config.get("pre_market_gap_percentage_threshold", 0)
    pre_market_vmap_drawdown_threshold = config.get("pre_market_vmap_drawdown_threshold", 0)
    # 3b. Apply the filters
    filtered_df = filter_by_pre_market_change(df, pre_market_change_pct_threshold)
    filtered_df = filter_by_float_shares(filtered_df, float_shares_threshold)
    filtered_df = filter_by_relative_volume(filtered_df, relative_volume_threshold)
    filtered_df = filter_by_relative_volume_at_time(filtered_df, relative_volume_at_time_threshold)
    filtered_df = filter_by_volatility(filtered_df) # SPECIAL FILTER
    # filtered_df = filter_by_price_above_vwap(filtered_df) # SPECIAL FILTER
    filtered_df = filter_by_pre_market_gap_percentage(filtered_df, pre_market_gap_percentage_threshold)
    filtered_df = filter_by_price_near_vwap(filtered_df, pre_market_vmap_drawdown_threshold)
  
    # 3c. Return the filtered DataFrame
    return filtered_df

In [517]:
#EXECUTE
tv_L2_df = pd.DataFrame()
categories = tv_df['marketCapType'].unique()

for category in categories:
    category_df = tv_df[tv_df['marketCapType'] == category]
    screened_df = screen_stocks_by_category(category_df, category)
    tv_L2_df = pd.concat([tv_L2_df, screened_df])

# Display results
print(f'Original DataFrame length: {len(tv_df.index)}')
print(f'Number of rows removed: {len(tv_df.index) - len(tv_L2_df.index)}')
print(f'NEW DataFrame length: {len(tv_L2_df.index)}')

# filter in descending order by market cap and then by pre-market change percentage
tv_L2_df = tv_L2_df.sort_values(by=['Market capitalization', 'Pre-market Change %'], ascending=[False, False])
# reset index and drop
tv_L2_df = tv_L2_df.reset_index(drop=True)

Original DataFrame length: 824
Number of rows removed: 814
NEW DataFrame length: 10


In [518]:
# create a breakdown of the market cap types
market_cap_type_df3 = tv_L2_df['marketCapType'].value_counts().reset_index()
market_cap_type_df3.columns = ['marketCapType', 'count']  # Rename columns for clarity
# calculate percentage for each market cap type and add it to the DataFrame
market_cap_type_df3['percentage'] = (market_cap_type_df3['count'] / market_cap_type_df3['count'].sum()) * 100

print('Returned Gap-Up Market Cap Type Groups:')
market_cap_type_df3

Returned Gap-Up Market Cap Type Groups:


,marketCapType,count,percentage
0,Large caps,4,40.0
1,Small caps,4,40.0
2,Midlers,2,20.0


In [519]:
# market cap type groups
print(f"Market Cap Groups:\n{tv_L2_df['marketCapType'].unique()}\n")

# TradingView column names
tv_raw_columns_list = tv_L2_df.columns.tolist()
print(f'TradingView Column Names (raw data):\n{tv_raw_columns_list}\n')

# stocks returned from the pre-market gap up screener
print(f'{len(tv_L2_df.index)} = U.S. stocks returned from the pre-market gap up screener.')

# create a list of 'Symbols' from the tv_L2_df
gap_up_stocks_list = tv_L2_df['Symbol']
gap_up_stocks_list = gap_up_stocks_list.tolist()
print(gap_up_stocks_list)

# FILTERD DATAFRAME WITH GAP-UP STOCK TARGETS
tv_L2_df


Market Cap Groups:
['Large caps' 'Midlers' 'Small caps']

TradingView Column Names (raw data):
['Symbol', 'Description', 'Exchange', 'Market capitalization', 'Market capitalization - Currency', 'Price', 'Price - Currency', 'Pre-market Open', 'Pre-market Open - Currency', 'Pre-market Change', 'Pre-market Change - Currency', 'Pre-market Change %', 'Pre-market Gap %', 'Float shares outstanding', 'Volume 1 day', 'Volume 1 week', 'Pre-market Volume', 'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days', 'Volatility 1 day', 'Volatility 1 week', 'Volatility 1 month', 'Volume Weighted Average Price 1 day', 'Price to earnings ratio', 'Sector', 'Relative Volume at Time', 'Beta 1 year', 'Beta 3 years', 'Beta 5 years', 'Relative Volume 1 minute', 'Relative Volume 5 minutes', 'Relative Volume 15 minutes', 'Relative Volume 30 minutes', 'Relative Volume 1 hour', 'Relative Volume 2 hours', 'Relative Volume 4 hours', 'Relative Volume 1 day', 'Relative Volume 1 week', 'Relative V

,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,BLK,"BlackRock, Inc.",NYSE,1.242880e+11,USD,834.47,USD,836.58,USD,2.12,USD,0.254053,0.252855,1.464672e+08,858869,858869,283,573983.0,604220.43333333,643142.33333333,3.324264,1.839686,1.478662,830.798333,22.85067254,Finance,3.747127,1.6802136,1.5869129,1.3833853,3.449215,7.026126,4.205254,2.292797,1.284237,0.806987,1.056920,1.628269,0.32260495,0.09892093,841.9550,USD,841.955,USD,841.955,USD,841.9550,USD,973.1600,USD,834.735,USD,836.93,USD,839.5000,USD,839.5000,USD,839.5000,USD,841.9550,USD,841.955,USD,841.955,USD,841.955,USD,942222.22222222,USD,834.1090,834.336675,835.91100,838.555769,838.552893,835.782595,833.012297,843.319646,839.032250,834.744854,Large caps
1,COIN,"Coinbase Global, Inc.",NASDAQ,5.551736e+10,USD,229.15,USD,232.98,USD,3.54,USD,1.544840,1.671394,1.688829e+08,23294868,23294868,633870,16260449.2,13916867.86666667,13597284.9,14.459575,8.470477,7.409201,225.952300,850.27829314,Finance,2.364925,2.7703786,3.3664243,3.3664243,2.841753,2.678537,2.218892,1.554079,0.940993,0.605841,0.742158,1.350091,0.3394343,0.12231877,236.4600,USD,236.460,USD,236.460,USD,236.4600,USD,429.5400,USD,230.080,USD,231.59,USD,235.3972,USD,235.3972,USD,235.3972,USD,236.4600,USD,236.460,USD,236.460,USD,236.460,USD,909948.18501171,USD,229.3140,229.212762,229.67498,232.722677,234.280743,230.057150,225.833557,237.833015,232.852495,227.871975,Large caps
2,FNV,Franco-Nevada Corporation,NYSE,2.103131e+10,USD,109.47,USD,110.50,USD,1.13,USD,1.032246,0.940897,1.906962e+08,1331162,1331162,1000,754124.1,664911.2,792113.4,2.680990,2.440765,1.947856,108.793333,31.11998975,Non-Energy Minerals,3.628591,0.24055775,0.73277336,0.58878154,6.397003,10.000397,8.974410,3.729317,1.627670,1.053902,1.083786,1.945390,0.44419358,0.21227832,110.8200,USD,113.980,USD,145.820,USD,161.2500,USD,169.3200,USD,109.825,USD,109.86,USD,109.8600,USD,109.8600,USD,109.8600,USD,109.8600,USD,109.860,USD,109.860,USD,109.860,USD,28111311.19470551,USD,109.4560,109.598750,109.59800,109.269231,109.907373,109.611500,109.315627,109.813001,109.063500,108.313999,Large caps
3,LOGI,Logitech International S.A. - Registered Shares,NASDAQ,1.448407e+10,USD,86.33,USD,86.86,USD,0.37,USD,0.428588,

| Gap-Up Screener |
|--|
IEX Cloud API (GET Request)

In [520]:
"""
A.
THIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))
-try to get foreign news where people are awake
-translation (internationalization) feature would be both advanced as well as practical
-all info about these companies so we start to learn what we are wokring with today
-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade

B.
-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.
-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?

C.
-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)

"""

"\nA.\nTHIS IS ALSO GOING TO BE YOUR NEWS, CURRENT NEWS (like almost real-time (within 15-30mins))\n-try to get foreign news where people are awake\n-translation (internationalization) feature would be both advanced as well as practical\n-all info about these companies so we start to learn what we are wokring with today\n-maybe some fundamentals although less important for gap up? unless gap up turns into swing trade\n\nB.\n-need a closeness list. some rating system. avoid if a stock barely misses one or two filter metrics.\n-so it's two part, 1-the filter closeness boolean as well as well has how close this miss was in terms of percentage and like an aggregate or miss difference -> THEN scoring system?\n\nC.\n-you are going to need also a -> 'chart_tv_df (this will be for the TradingView data)\n\n"

In [548]:
"""NEWS AND THAT INTERNATIONALIZATION SHIT GOES HERE"""
info_news_df = pd.DataFrame(tv_L2_df, columns=['Symbol', 'Exchange', 'marketCapType'])

info_news_df

,Symbol,Exchange,marketCapType
0,BLK,NYSE,Large caps
1,COIN,NASDAQ,Large caps
2,FNV,NYSE,Large caps
3,LOGI,NASDAQ,Large caps
4,APGE,NASDAQ,Midlers
5,PAY,NYSE,Midlers
6,GRND,NYSE,Small caps
7,VVX,NYSE,Small caps
8,NGNE,NASDAQ,Small caps
9,VBNK,NASDAQ,Small caps


In [522]:
# # manual add list
# manual_add_list = ['DELL','PLTR','TSLA']
# # append list with manually added stocks
# gap_up_stocks_list.extend(manual_add_list)

# print(f'{manual_add_list} added manually\n')
# print(f'New list: {gap_up_stocks_list}')

In [523]:
#### IEX CLOUD API KEY -> VARIABLE ####
iex_api_key = os.getenv("IEX_API_KEY")

# set the list of symbols variable
symbols = ','.join(gap_up_stocks_list)

# SETTING THE DATE RANGE
"""
y - Example: range=1y -> returns (one year)
ytd - Example: range=ytd -> returns (year-to-date)
m - Example: range=6m returns (six months)
d - Example: range=5d returns (five days)
"""
data_range = '4m' # DATE RANGE (for historical stock price data)

url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=chart&range={data_range}&token={iex_key}'

# Make the GET request
response = requests.get(url)

# Initialize a list to collect DataFrames for each symbol
dataframes_list = []

# Check if the request was successful
if response.status_code == 200:
    # Convert the response to JSON
    data = response.json()

    # Process and print the data with a progress bar
    for symbol in tqdm(gap_up_stocks_list, desc="Processing symbols"):
        if symbol in data:
            # Create a DataFrame for the current symbol's data
            symbol_df = pd.DataFrame(data[symbol]['chart'])
            # Add a 'Ticker' column to identify the data
            symbol_df['Ticker'] = symbol
            # Append the current symbol's DataFrame to the list
            dataframes_list.append(symbol_df)
            # print(f'Data for {symbol}:')
            # print(symbol_df.head(5))  # print the first few rows of the DataFrame for this symbol
else:
    print(f'Failed to fetch data: {response.status_code}')

# concatenate dataframes and create a combined list
historical_df = pd.concat(dataframes_list, ignore_index=True)


Processing symbols: 100%|██████████| 10/10 [00:00<00:00, 1047.79it/s]


In [524]:
# Set variables for historical data analysis
historical_length = len(historical_df.index)
unique_stocks = historical_df['Ticker'].nunique()
start_date = historical_df['date'].min()
end_date = historical_df['date'].max()

print(f"START date: {start_date}")
print(f"END date: {end_date}")
print(f'{historical_df["date"].nunique()} days of historical stock data.\n')
print(f"{historical_length} = Length of dataframe")
print(f"{unique_stocks} = Unique stocks in dataframe")

# Entries per stock
days_per_stock = historical_length / unique_stocks
print(f"{days_per_stock} = Date entries per stock (MATH check -> should be the same as the unique )\n")

# View the stocks
print(f'Unique stock ticker symbols: {gap_up_stocks_list}')
historical_cols_list = historical_df.columns.tolist()
print(f'Columns: {historical_cols_list}')

# convert df to .csv
historical_df.to_csv(f'historical_stock_data{today_date_str}.csv', index=False)

# Display the first few rows of the dataframe
historical_df.head()

START date: 2023-11-06
END date: 2024-03-04
81 days of historical stock data.

809 = Length of dataframe
10 = Unique stocks in dataframe
80.9 = Date entries per stock (MATH check -> should be the same as the unique )

Unique stock ticker symbols: ['BLK', 'COIN', 'FNV', 'LOGI', 'APGE', 'PAY', 'GRND', 'VVX', 'NGNE', 'VBNK']
Columns: ['close', 'high', 'low', 'open', 'priceDate', 'symbol', 'volume', 'id', 'key', 'subkey', 'date', 'updated', 'changeOverTime', 'marketChangeOverTime', 'uOpen', 'uClose', 'uHigh', 'uLow', 'uVolume', 'fOpen', 'fClose', 'fHigh', 'fLow', 'fVolume', 'label', 'change', 'changePercent', 'Ticker']


,close,high,low,open,priceDate,symbol,volume,id,key,subkey,date,updated,changeOverTime,marketChangeOverTime,uOpen,uClose,uHigh,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent,Ticker
0,653.22,660.255,650.355,656.62,2023-11-06,BLK,469385.0,HISTORICAL_PRICES,BLK,,2023-11-06,1702073971000,0.000000,0.000000,656.62,653.22,660.255,650.355,469385,652.268,648.890,655.878,646.044,469385.0,"Nov 6, 23",0.00,0.0000,BLK
1,648.59,653.500,646.905,652.82,2023-11-07,BLK,440812.0,HISTORICAL_PRICES,BLK,,2023-11-07,1702073975000,-0.007088,-0.007088,652.82,648.59,653.500,646.905,440812,648.493,644.291,649.168,642.617,440812.0,"Nov 7, 23",-4.63,-0.0071,BLK
2,654.95,655.660,648.430,648.43,2023-11-08,BLK,482206.0,HISTORICAL_PRICES,BLK,,2023-11-08,1702073986000,0.002648,0.002648,648.43,654.95,655.660,648.430,482206,644.132,650.609,651.314,644.132,482206.0,"Nov 8, 23",6.36,0.0098,BLK
3,651.82,660.600,650.630,656.73,2023-11-09,BLK,437166.0,HISTORICAL_PRICES,BLK,,2023-11-09,1702074009000,-0.002143,-0.002143,656.73,651.82,660.600,650.630,437166,652.377,647.499,656.221,646.317,437166.0,"Nov 9, 23",-3.13,-0.0048,BLK
4,665.00,665.450,653.876,656.58,2023-11-10,BLK,522796.0,HISTORICAL_PRICES,BLK,,2023-11-10,1702073999000,0.018034,0.018034,656.58,665.00,665.450,653.876,522796,652.228,660.592,661.039,649.542,522796.0,"Nov 10, 23",13.18,0.0202,BLK


In [525]:
#### DEV - DATA ORGANIZATION ####
# convert df to .csv
historical_df.to_csv(f'/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/trading_view_raw_data/historical_stock_data{today_date_str}.csv', index=False)

In [526]:
# Ensure the dataframe is sorted by date if it's not already
historical_sorted_df = historical_df.sort_values(by='date')

# Select mandatory columns for a historical price bar chart + volume
chart_historical_df = historical_sorted_df[[
    'symbol',   # *MANDATORY
    'date',     # *MANDATORY
    'close',    # *MANDATORY
    'high',     # *MANDATORY
    'low',      # *MANDATORY
    'open',     # *MANDATORY
    'volume'    # *MANDATORY
]]

# Group by 'symbol' and get the last occurrence
last_date_df = chart_historical_df.groupby('symbol').last().reset_index()

last_date_df

,symbol,date,close,high,low,open,volume
0,APGE,2024-03-04,41.74,43.250,37.7800,37.9000,1372617.0
1,BLK,2024-03-04,834.47,841.955,815.9700,815.9700,858869.0
2,COIN,2024-03-04,229.15,236.460,212.2469,217.3901,23294868.0
3,FNV,2024-03-04,109.47,109.860,107.0500,108.4500,1331162.0
4,GRND,2024-03-04,8.33,8.350,7.9300,8.1100,589845.0
5,LOGI,2024-03-04,86.33,87.040,84.8900,87.0400,630685.0
6,NGNE,2024-03-04,38.55,39.950,32.0000,32.8100,299772.0
7,PAY,2024-03-04,16.35,16.950,15.8500,16.6700,1092398.0
8,VBNK,2024-03-04,11.71,12.040,11.2520,12.0400,57263.0
9,VVX,2024-03-04,38.64,39.350,38.2681,39.0100,106015.0


In [547]:
# re-index the dataframe
chart_historical_df.reset_index(drop=True, inplace=True)
# view head of your main df
print(f'Length of historical data: {len(chart_historical_df.index)}')

Length of historical data: 809


In [566]:
"""
YOUR DATAFRAMES TO MOVE FORWARD WITH:
"""
#### INFO / NEWS DATAFRAME ####
print(f'1 - NEWS & STOCK INFO (trending news etc.):\n{info_news_df}\n')

#### IEX (API) - HISTORICAL DATA FRAME ####
print('2a - IEX HISTORICAL data (START):')
print(f'{chart_historical_df.head()}\n')
print('2b - IEX HISTORICAL data (END):')
print(f'{chart_historical_df.tail()}\n')

1 - NEWS & STOCK INFO (trending news etc.):
  Symbol Exchange marketCapType
0    BLK     NYSE    Large caps
1   COIN   NASDAQ    Large caps
2    FNV     NYSE    Large caps
3   LOGI   NASDAQ    Large caps
4   APGE   NASDAQ       Midlers
5    PAY     NYSE       Midlers
6   GRND     NYSE    Small caps
7    VVX     NYSE    Small caps
8   NGNE   NASDAQ    Small caps
9   VBNK   NASDAQ    Small caps

2a - IEX HISTORICAL data (START):
  symbol        date   close     high      low     open      volume
0    BLK  2023-11-06  653.22  660.255  650.355  656.620   469385.00
1   NGNE  2023-11-06   13.68   13.800   13.060   13.680     1638.25
2    VVX  2023-11-06   40.73   48.960   40.240   47.830   184616.00
3   VBNK  2023-11-06    7.65    7.830    7.650    7.650     2992.00
4   COIN  2023-11-06   86.37   89.270   84.090   86.815  9881917.00

2b - IEX HISTORICAL data (END):
    symbol        date   close     high       low      open      volume
804    FNV  2024-03-04  109.47  109.860  107.0500  108.4

In [565]:
#### TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY'S DATA) ####
print('3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):')
tv_L2_df

3 - TRADINGVIEW (IMPORTED) - DATAFRAME (TODAY):


,Symbol,Description,Exchange,Market capitalization,Market capitalization - Currency,Price,Price - Currency,Pre-market Open,Pre-market Open - Currency,Pre-market Change,Pre-market Change - Currency,Pre-market Change %,Pre-market Gap %,Float shares outstanding,Volume 1 day,Volume 1 week,Pre-market Volume,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Volatility 1 day,Volatility 1 week,Volatility 1 month,Volume Weighted Average Price 1 day,Price to earnings ratio,Sector,Relative Volume at Time,Beta 1 year,Beta 3 years,Beta 5 years,Relative Volume 1 minute,Relative Volume 5 minutes,Relative Volume 15 minutes,Relative Volume 30 minutes,Relative Volume 1 hour,Relative Volume 2 hours,Relative Volume 4 hours,Relative Volume 1 day,Relative Volume 1 week,Relative Volume 1 month,New high 1 month,New high 1 month - Currency,New high 3 months,New high 3 months - Currency,New high 6 months,New high 6 months - Currency,New high 52 weeks,New high 52 weeks - Currency,New high All Time,New high All Time - Currency,High 5 minutes,High 5 minutes - Currency,High 15 minutes,High 15 minutes - Currency,High 30 minutes,High 30 minutes - Currency,High 1 hour,High 1 hour - Currency,High 2 hours,High 2 hours - Currency,High 4 hours,High 4 hours - Currency,High 1 day,High 1 day - Currency,High 1 week,High 1 week - Currency,High 1 month,High 1 month - Currency,"Revenue per employee, Annual","Revenue per employee, Annual - Currency",Simple Moving Average (5) 1 minute,Simple Moving Average (8) 1 minute,Simple Moving Average (5) 5 minutes,Simple Moving Average (13) 5 minutes,"Bollinger Bands (20) 1 minute, Upper","Bollinger Bands (20) 1 minute, Basis","Bollinger Bands (20) 1 minute, Lower","Bollinger Bands (20) 5 minutes, Upper","Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",marketCapType
0,BLK,"BlackRock, Inc.",NYSE,1.242880e+11,USD,834.47,USD,836.58,USD,2.12,USD,0.254053,0.252855,1.464672e+08,858869,858869,283,573983.0,604220.43333333,643142.33333333,3.324264,1.839686,1.478662,830.798333,22.85067254,Finance,3.747127,1.6802136,1.5869129,1.3833853,3.449215,7.026126,4.205254,2.292797,1.284237,0.806987,1.056920,1.628269,0.32260495,0.09892093,841.9550,USD,841.955,USD,841.955,USD,841.9550,USD,973.1600,USD,834.735,USD,836.93,USD,839.5000,USD,839.5000,USD,839.5000,USD,841.9550,USD,841.955,USD,841.955,USD,841.955,USD,942222.22222222,USD,834.1090,834.336675,835.91100,838.555769,838.552893,835.782595,833.012297,843.319646,839.032250,834.744854,Large caps
1,COIN,"Coinbase Global, Inc.",NASDAQ,5.551736e+10,USD,229.15,USD,232.98,USD,3.54,USD,1.544840,1.671394,1.688829e+08,23294868,23294868,633870,16260449.2,13916867.86666667,13597284.9,14.459575,8.470477,7.409201,225.952300,850.27829314,Finance,2.364925,2.7703786,3.3664243,3.3664243,2.841753,2.678537,2.218892,1.554079,0.940993,0.605841,0.742158,1.350091,0.3394343,0.12231877,236.4600,USD,236.460,USD,236.460,USD,236.4600,USD,429.5400,USD,230.080,USD,231.59,USD,235.3972,USD,235.3972,USD,235.3972,USD,236.4600,USD,236.460,USD,236.460,USD,236.460,USD,909948.18501171,USD,229.3140,229.212762,229.67498,232.722677,234.280743,230.057150,225.833557,237.833015,232.852495,227.871975,Large caps
2,FNV,Franco-Nevada Corporation,NYSE,2.103131e+10,USD,109.47,USD,110.50,USD,1.13,USD,1.032246,0.940897,1.906962e+08,1331162,1331162,1000,754124.1,664911.2,792113.4,2.680990,2.440765,1.947856,108.793333,31.11998975,Non-Energy Minerals,3.628591,0.24055775,0.73277336,0.58878154,6.397003,10.000397,8.974410,3.729317,1.627670,1.053902,1.083786,1.945390,0.44419358,0.21227832,110.8200,USD,113.980,USD,145.820,USD,161.2500,USD,169.3200,USD,109.825,USD,109.86,USD,109.8600,USD,109.8600,USD,109.8600,USD,109.8600,USD,109.860,USD,109.860,USD,109.860,USD,28111311.19470551,USD,109.4560,109.598750,109.59800,109.269231,109.907373,109.611500,109.315627,109.813001,109.063500,108.313999,Large caps
3,LOGI,Logitech International S.A. - Registered Shares,NASDAQ,1.448407e+10,USD,86.33,USD,86.86,USD,0.37,USD,0.428588,

In [ ]:
"""PICK UP HERE WHERE YOU LEFT OFF"""

# DELL FOCUS - Pre-market

In [528]:
# # $DELL
# dell_df = key_levels_df[key_levels_df['symbol'] == 'DELL'].copy()
# dell_df['date'] = pd.to_datetime(dell_df['date'])
# dell_df.set_index('date', inplace=True)
# dell_df.head(5)

In [529]:
# import mplfinance as mpf
# # candlestick chart
# # MANDATORY DATA = 'open', 'high', 'low', 'close', 'volume'

# """
# Part 1:    
# """
# # Part 1a: SMAs -> calculate
# dell_df['SMA20'] = dell_df['close'].rolling(window=20, min_periods=1).mean()
# dell_df['SMA50'] = dell_df['close'].rolling(window=50, min_periods=1).mean()
# # dell_df['SMA200'] = dell_df['close'].rolling(window=200, min_periods=1).mean()

# # Part 1b: VWAP -> calculate
# dell_df['VWAP'] = (dell_df['volume'] * (dell_df['high'] + dell_df['low'] + dell_df['close']) / 3).cumsum() / dell_df['volume'].cumsum()

# """   
# Part 2: 
# """
# add2plot = [
#     #### SMAs ####
#     mpf.make_addplot(dell_df['SMA20'], color='#7BAFD4', width=1.0, label='SMA 20'), # UNC blue
#     mpf.make_addplot(dell_df['SMA50'], color='#003087', width=1.0, label='SMA 50'), # DUKE blue
#     # mpf.make_addplot(dell_df['SMA200'], color='#041E42', width=1.0, label='SMA 200'), # HOYAS blue
#     #### VWAP ####
#     mpf.make_addplot(dell_df['VWAP'], color='#B31B1B', width=1.0, label='VWAP') # BIG RED
# ]

# """   
# Part 3: styling not iterative changes related to data (i.e., color, font, etc.)
# """
# # Use a dark theme style and set text color to white
# mpf_style = mpf.make_mpf_style(base_mpf_style='nightclouds', rc={'text.color': 'white'})

# # Plot the candlestick chart with the moving averages
# fig, axes = mpf.plot(dell_df,
#                      type='candle',
#                      addplot=add2plot,
#                      volume=True,
#                     #  style=mpf_style,
#                      style=s, 
#                      # WTF look at docs (it's not erroring out but going to default style just figure out what that is i think)
#                      figratio=(15, 8),
#                      title='$DELL - Historical Data (6 Months)',
#                      tight_layout=True,
#                      returnfig=True)

# # Add a watermark if needed
# fig.text(0.5, 0.5, 'DELL', fontsize=50, color='gray', alpha=0.5, ha='center', va='center', rotation=0, transform=fig.transFigure)

# plt.show()


In [530]:
# dell_df.head(5)

In [531]:
""" 
ISSUE: some weird shit with the style param
nbd tho ctfo. 

workaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  
the blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style
experienced similar shit like this years ago, idunno will get to it.

"""

' \nISSUE: some weird shit with the style param\nnbd tho ctfo. \n\nworkaround =  is first successfully run with style=mpf_style, then swap it for style=s (dark mode - but common view like Ortex)  \nthe blue shit and mpf_style is actually cool but might confuse user (i.e., blue = red and white = green) with the mpf_style\nexperienced similar shit like this years ago, idunno will get to it.\n\n'

In [532]:
# you're going to need different charts, one for the pre-market, and at least another for market open
# I'd like market opens to be a live feed but really we might already be in trading view watching the pine script execute

# Trend lines

| END OF IN PROGRESS WORK - NOTES BELOW |
|--|

In [533]:
# but first start with getting lines on that chart with STRV
# let's get those key levels with an output on them and figure out your strategy
# figure out when fibonacci extensions get activated
# i see one view with a grid layout of the stock charts above or below them the summary with the key levels, current price and all that
# i also see a view where when looking at individiual stocks we got data all over that shit. idgaf if it's "confusing for the user"

# you have your key_levels_df
# so start next from there okay

#### </b> ####

# make title more symetrically bigger not just bigger for fucks sake 
# move price to the right side, keep vol on left
# see if you can get a marker or military like custom google font one of the ones you like
# better yet use that program you wrote for sn logos actually to create your shit
# could put that Sans Peur Systems somewhere at least watermark it, maybe?
# key levels find up to 6?, scoring strategy?, then base allotment of position (1/3, 1/5) conditionally?
# we could add RSI down there, fuck could add ichimoku, do whatever with all this data access now

NOTES -> SMA Legend

In [534]:
# # SMA20, SMA50, and SMA200 has to be in 'dell_df'
# legend_info = {
#     'Indicator': ['SMA20', 'SMA50', 'SMA200'],
#     'Color': ['blue', 'orange', 'green'],
#     'Description': ['20-day Moving Average', '50-day Moving Average', '200-day Moving Average']
# }

# legend_df = pd.DataFrame(legend_info)
# print(legend_df)


NOTES -> custom mplfinance chart setup example

In [535]:
# """    
# Custom Theme setup
# """

# dark_style = mpf.make_mpf_style(base_mpf_style='charles', 
#                                 rc={'axes.facecolor': 'black',
#                                     'figure.facecolor': 'darkslategrey',
#                                     'axes.grid': True,
#                                     'grid.color': 'gray',
#                                     'grid.linestyle': '--',
#                                     'grid.linewidth': 0.6,
#                                     'axes.titlesize': 16,
#                                     'axes.titleweight': 'bold',
#                                     'axes.labelsize': 12,
#                                     'axes.labelweight': 'bold',
#                                     'axes.labelcolor': 'white',
#                                     'axes.edgecolor': 'white',
#                                     'axes.linewidth': 2,
#                                     'xtick.color': 'white',
#                                     'xtick.labelsize': 10,
#                                     'ytick.color': 'white',
#                                     'ytick.labelsize': 10,
#                                     'figure.titlesize': 18,
#                                     'figure.titleweight': 'bold',
#                                     'legend.facecolor': 'grey',
#                                     'legend.edgecolor': 'white',
#                                     'legend.fontsize': 10,
#                                     'legend.title_fontsize': 12})

# mpf.plot(strv_df, 
#          type='candle', 
#          style=dark_style,  # Using the custom dark style
#          volume=True, 
#          title='STRV Candlestick Chart',
#          mav=(20, 50, 200),  # Adding moving averages of 20, 50, and 200 periods
#          figratio=(15, 8), 
#          tight_layout=True)

In [536]:
# i need a math checker for trading days vs calendar days (252/365)

Notes -> yahoo finance API exaple (free) *with limitations

In [537]:
# import yfinance as yf
# from datetime import datetime, timedelta

In [538]:
# # yahoo finance data download
# start = datetime.now() - timedelta(days=60)
# end = datetime.now()
# yahoo_df = yf.download(gap_up_stocks_list, start, end)
# yahoo_df = yahoo_df.loc[:,'Close']
# # view the yahoo data yahoo finance formatting multi-index (has a 60day max with standard API maneuvers)
# yahoo_df.head(10)